In [2]:
print("\n🏗️  Society of Mind Architecture:")
print("""
    ┌─────────────────────────────────────-┐
    │        Society of Mind Agent         │
    │  ┌─────────────────────────────────┐ │
    │  │         Inner Team              │ │
    │  │  ┌─────────┐  ┌─────────────┐   │ │
    │  │  │ Agent A │  │   Agent B   │   │ │
    │  │  │(Writer) │  │  (Editor)   │   │ │
    │  │  └─────────┘  └─────────────┘   │ │
    │  │         │            │          │ │
    │  │         └────┬───────┘          │ │
    │  │              │                  │ │
    │  │         Discussion              │ │
    │  └─────────────────────────────────┘ │
    │                 │                    │
    │            Synthesis                 │
    │                 │                    │
    │           Final Response             │
    └─────────────────────────────────────-┘
    """)


🏗️  Society of Mind Architecture:

    ┌─────────────────────────────────────-┐
    │        Society of Mind Agent         │
    │  ┌─────────────────────────────────┐ │
    │  │         Inner Team              │ │
    │  │  ┌─────────┐  ┌─────────────┐   │ │
    │  │  │ Agent A │  │   Agent B   │   │ │
    │  │  │(Writer) │  │  (Editor)   │   │ │
    │  │  └─────────┘  └─────────────┘   │ │
    │  │         │            │          │ │
    │  │         └────┬───────┘          │ │
    │  │              │                  │ │
    │  │         Discussion              │ │
    │  └─────────────────────────────────┘ │
    │                 │                    │
    │            Synthesis                 │
    │                 │                    │
    │           Final Response             │
    └─────────────────────────────────────-┘
    


In [5]:
from dotenv import load_dotenv
load_dotenv()
import os 
api_key = os.getenv('OPENAI_API_KEY')


In [6]:
import asyncio
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import AssistantAgent, SocietyOfMindAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination


async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o",api_key=api_key)

    agent1 = AssistantAgent("assistant1", model_client=model_client, system_message="You are a writer, write well under 50 words.")
    agent2 = AssistantAgent(
        "assistant2",
        model_client=model_client,
        system_message="You are an editor, provide critical feedback under 50 words. Respond with 'APPROVE' if the text addresses all feedbacks.",
    )
    inner_termination = TextMentionTermination("APPROVE")
    inner_team = RoundRobinGroupChat([agent1, agent2], termination_condition=inner_termination)


    society_of_mind_agent = SocietyOfMindAgent("society_of_mind", team=inner_team, model_client=model_client,response_prompt='Output a standalone response to the original request under 50 words, without mentioning any of the intermediate discussion.')

   # agent4 = society_of_mind_agent = SocietyOfMindAgent("society_of_mind", team=inner_team, model_client=model_client,response_prompt='Output a standalone response to the original request under 50 words, without mentioning any of the intermediate discussion.')





    agent3 = AssistantAgent(
        "assistant3", model_client=model_client, system_message="Translate the text to Spanish under 50 words."
    )
    team = RoundRobinGroupChat([society_of_mind_agent, agent3], max_turns=2)

    stream = team.run_stream(task="Write a short story with a surprising ending under 50 words.")
    await Console(stream)


await (main())

---------- TextMessage (user) ----------
Write a short story with a surprising ending under 50 words.
---------- TextMessage (assistant1) ----------
In the bustling city, Jane followed a stranger's dropped wallet. Returning it, she was startled to see her own photo inside. The stranger smiled, "I've been looking for you." Jane realized she was meeting her twin, separated at birth. Their unexpected reunion was a twist of fate.
---------- TextMessage (assistant2) ----------
Your story captures a surprise ending well, but it subtly hints at the twist too early. Consider obscuring the connection between Jane and the stranger more until the last moment for a stronger impact. Overall, it's engaging and concise. 

APPROVE
---------- TextMessage (society_of_mind) ----------
In the city, Jane returns a stranger's wallet, only to find her photo inside. The stranger reveals they are long-lost twins, meeting unexpectedly.
---------- TextMessage (assistant3) ----------
En la ciudad, Jane devuelve l